In [ ]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


from datetime import datetime
import time
import datetime

import eikon as ek
#creds = 'f5a341b614d945b9b93a71fe53348ba119d11e99'
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
#API Key Laurin:
ek.set_app_key('d505f265d9bd4c0cb7df44481ece31a0ad8d1974')

GDP nicht nehmen, da nur Quartal / unemployment rate / Inflation (CPI) / purchasing manager index (PMI)

US / Eurozone / CH / China

In [ ]:
Makro_Data = ['USUNR=ECI',  #US Unemploymet
              'USCPI=ECI',  #US CPI
              'USPMI=ECI',  #US PMI
              
              'EUUNR=ECI',  #EU Unemployment
              'EUHICY=ECI', #EU CPI
              'EUPMI=ECI',  #EU PMI                NICHT GUT (Nicht vollständig)
              
              'CHJOB=ECI',  #CH Unemployment
              'CHCPIY=ECI', #CH CPI
              'CHPMI=ECI',  #CH PMI
              
            #China is in the Dataset due to its exposure to the Metals

              'CNURUA=ECI', #China Unemployment    NICHT GUT (Nicht vollständig)
              'CNCPI=ECI',   #China CPI
              'CNPMIB=ECI',  #China PMI

            #Russia is in the Dataset due to its exporsure to Oil & Gas

              'RUUNR=ECI',  #Russia Unemployment
              'RUCPIY=ECI', #Russia CPI
              'RUPMIM=ECI', #Russia PMI            NICHT GUT (Nicht vollständig)
             ]  

In [ ]:
macro = ek.get_timeseries(Makro_Data,
                          start_date='2012-01-01',
                          end_date='2022-01-01',
                          fields='VALUE',
                          interval='monthly')                      

In [ ]:
macro.isnull().sum()

In [ ]:
macro.describe()

In [ ]:
macro.plot(subplots=True, figsize=(15,15))

In [ ]:
macro2 = macro
macro2.columns = macro2.columns.str.replace("=", "_")
macro2 = macro2.unstack()
macro2 = macro2.reset_index()
macro2 = macro2.rename(columns={'VALUE': 'Macro_Variable', 0: 'Value'})
macro2['Date'] = pd.to_datetime(macro2['Date']).dt.date
liste_SQL = macro2.values.tolist()
macro2.head(5)

In [ ]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

In [ ]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS MACRO_DATA;''')

In [ ]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''CREATE TABLE MACRO_DATA (
    "Macro_Variable" TEXT,
    DATE DATETIME,
	"Value" NUMERIC
);''')


In [ ]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")
#Add all Data at once:
c = conn.cursor()
c.executemany("INSERT INTO MACRO_DATA(Macro_Variable, DATE, Value) VALUES (?,?,?)", liste_SQL)    
conn.commit()

In [ ]:
#Select Data from SQL DB:   
c = conn.cursor()
c.execute("SELECT * FROM MACRO_DATA")
new_data=c.fetchall()
ref_data = pd.DataFrame(new_data,columns=['Macro_Variable', 'DATE', 'Value'])
ref_data = ref_data.set_index('DATE')
ref_data

In [ ]:
macro_data = ref_data.pivot(columns='Macro_Variable', values=['Value'])
macro_data.to_excel('Macro_Data_from_SQL.xlsx')
macro_data